In [60]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


class AnalizadorDatos:
    """
    Clase para el análisis de datos de un archivo .npy.
    Permite cargar los datos, dividirlos en entrenamiento y validación,
    calcular estadísticas descriptivas y generar histogramas con métricas.
    """

    def __init__(self, ruta_archivo):
        """
        Inicializa el analizador cargando los datos desde un archivo .npy.

        Parámetros:
        ruta_archivo (str): Ruta del archivo .npy con los datos.
        """
        self.ruta_archivo = ruta_archivo
        self.datos = None
        self.df = None
        self.entrenamiento = None
        self.validacion = None
        self.nombres_columnas = ["SalePrice", "OverallQual", "1stFlrSF", "TotRmsAbvGrd", "YearBuilt", "LotFrontage"]
        
        self.cargar_datos()
    
    def cargar_datos(self):
        """
        Carga los datos desde un archivo .npy, los convierte en un DataFrame de Pandas,
        y mezcla aleatoriamente las filas con una semilla fija para asegurar consistencia.
        """
        try:
            self.datos = np.load(self.ruta_archivo, allow_pickle=True)
            self.df = pd.DataFrame(self.datos, columns=self.nombres_columnas)

            # Mezclar las filas del DataFrame de forma reproducible
            self.df = self.df.sample(frac=1, random_state=42).reset_index(drop=True)

            # Tratar valores faltantes en LotFrontage
            if "LotFrontage" in self.df.columns:
                mediana = self.df["LotFrontage"].median()
                self.df["LotFrontage"].fillna(mediana, inplace=True)

            print("Datos cargados y mezclados correctamente.")
        except Exception as e:
            print(f"Error al cargar los datos: {e}")


    def dividir_datos(self, proporcion_entrenamiento=0.8):
        """
        Divide los datos mezclados en entrenamiento y validación.

        Parámetros:
        proporcion_entrenamiento (float): Proporción de los datos a utilizar para entrenamiento. 
                                        Por defecto es 0.8 (80%).
        """
        tamano_entrenamiento = int(len(self.df) * proporcion_entrenamiento)
        self.entrenamiento = self.df.iloc[:tamano_entrenamiento, :].copy()
        self.validacion = self.df.iloc[tamano_entrenamiento:, :].copy()
        print(f"Datos divididos en {tamano_entrenamiento} registros para entrenamiento y {len(self.df) - tamano_entrenamiento} para validación.")


    def calcular_estadisticas(self):
        """
        Calcula estadísticas descriptivas sobre los datos de entrenamiento.

        Retorna:
        pd.DataFrame: Un DataFrame con las estadísticas de cada columna, incluyendo:
                      - Media
                      - Máximo
                      - Mínimo
                      - Rango (peak-to-peak)
                      - Desviación estándar
        """
        if self.entrenamiento is None:
            print("Primero ejecuta dividir_datos() antes de calcular estadísticas.")
            return None
        
        estadisticas = {
            "Media": np.mean(self.entrenamiento, axis=0),
            "Maximo": np.max(self.entrenamiento, axis=0),
            "Minimo": np.min(self.entrenamiento, axis=0),
            "Rango": np.ptp(self.entrenamiento, axis=0),  # Peak-to-peak (max - min)
            "DesviacionEstandar": np.std(self.entrenamiento, axis=0)
        }
        
        df_estadisticas = pd.DataFrame(estadisticas, index=self.nombres_columnas)
        return df_estadisticas
    
    def mostrar_cabecera(self, n=5):
        """
        Muestra las primeras 'n' filas del DataFrame.

        Parámetros:
        n (int): Número de filas a mostrar. Por defecto es 5.
        """
        print(self.df.head(n))

    def graficar_histogramas(self):
        """
        Genera histogramas para cada variable en el dataset, incluyendo:
        - Media (línea roja)
        - Mediana (línea azul)
        - Desviación estándar (líneas verdes en ±1σ)
        - Máximo y mínimo (marcadores)
        """
        for columna in self.nombres_columnas:
            datos = self.df[columna].dropna()  # Elimina valores nulos si los hay

            # Calcular estadísticas
            media = np.mean(datos)
            mediana = np.median(datos)
            desviacion = np.std(datos)
            minimo = np.min(datos)
            maximo = np.max(datos)

            # Crear histograma con seaborn
            plt.figure(figsize=(8, 5))
            sns.histplot(datos, kde=True, bins=30, color='lightseagreen')
            #lightseagreen, turquoise

            # Agregar líneas de estadísticas
            plt.axvline(media, color='red', linestyle='dashed', linewidth=2, label=f'Media: {media:.2f}')
            plt.axvline(mediana, color='blue', linestyle='dashed', linewidth=2, label=f'Mediana: {mediana:.2f}')
            plt.axvline(media - desviacion, color='red', linestyle='dotted', linewidth=2, label=f'-1σ: {(media - desviacion):.2f}')
            plt.axvline(media + desviacion, color='orange', linestyle='dotted', linewidth=2, label=f'+1σ: {(media + desviacion):.2f}')

            # Marcar mínimo y máximo
            plt.scatter([minimo, maximo], [0, 0], color='black', zorder=3, label=f'Mín: {minimo:.2f}, Máx: {maximo:.2f}')

            # Configuración del gráfico
            plt.title(f"Histograma de {columna}")
            plt.xlabel(columna)
            plt.ylabel("Frecuencia")
            plt.legend()
            plt.grid(True)
            plt.show()

    def analizar_correlaciones(self):
        """
        Calcula y grafica la correlación entre cada variable independiente y la variable dependiente 'SalePrice'.
        Muestra un scatterplot por variable con su coeficiente de correlación en el título.
        Al final imprime las 2 variables con mayor correlación con 'SalePrice'.
        """
        if self.df is None:
            print("No hay datos cargados.")
            return

        variable_objetivo = "SalePrice"
        variables_independientes = [col for col in self.nombres_columnas if col != variable_objetivo]
        correlaciones = {}

        for variable in variables_independientes:
            x = self.df[variable]
            y = self.df[variable_objetivo]

            # Calcular correlación
            coef = np.corrcoef(x, y)[0, 1]
            correlaciones[variable] = coef

            # Graficar
            plt.figure(figsize=(7, 5))
            plt.scatter(x, y, alpha=0.6)
            plt.title(f"{variable} vs {variable_objetivo} (Correlación: {coef:.4f})")
            plt.xlabel(variable)
            plt.ylabel(variable_objetivo)
            plt.grid(True)
            plt.tight_layout()
            plt.show()

        # Ordenar correlaciones absolutas de mayor a menor
        correlaciones_ordenadas = sorted(correlaciones.items(), key=lambda item: abs(item[1]), reverse=True)

        print("Variables con mayor correlación con SalePrice:")
        for variable, coef in correlaciones_ordenadas[:2]:
            print(f"{variable}: {coef:.4f}")
            
    def graficar_matriz_correlacion_personalizada(self):
        """
        Genera una matriz de correlación personalizada:
        - Parte inferior: scatterplots.
        - Parte superior: coeficiente de correlación (Pearson).
        - Solo se usan seaborn, matplotlib, numpy, pandas.
        """
        # Definir dimensiones
        columnas = self.df.columns
        num_vars = len(columnas)

        # Crear PairGrid
        grid = sns.PairGrid(self.df, vars=columnas, diag_sharey=False)

        # Parte inferior: scatterplot
        grid.map_lower(sns.scatterplot, s=10, alpha=0.6)

        # Parte superior: texto con coeficiente de correlación
        def correlacion(x, y, **kwargs):
            r = np.corrcoef(x, y)[0, 1]
            ax = plt.gca()
            ax.annotate(f"{r:.2f}", xy=(0.5, 0.5), xycoords=ax.transAxes,
                        ha='center', va='center', fontsize=12)

        grid.map_upper(correlacion)

        # Diagonal: histogramas
        grid.map_diag(sns.histplot, kde=True)

        plt.tight_layout()
        plt.show()
        
    def entrenar_modelo_lineal_simple(self, x, y, epochs, imprimir_error_cada, alpha):
        """
        Entrena un modelo de regresión lineal simple y = β0 + β1 * x utilizando descenso por gradiente.

        Parámetros:
        x (np.ndarray): Variable independiente (6.1)
        y (np.ndarray): Variable dependiente (6.2)
        epochs (int): Número de iteraciones de entrenamiento (6.3)
        imprimir_error_cada (int): Frecuencia de impresión del error (6.4)
        alpha (float): Tasa de aprendizaje (6.5)

        Retorna:
        tuple: β0 (intercepto), β1 (pendiente)
        """
        n = len(x)

        # Inicialización de parámetros
        beta_0 = 0
        beta_1 = 0

        for epoch in range(1, epochs + 1):

            # 6.1 y 6.2: cálculo de predicciones usando x e y
            y_pred = beta_0 + beta_1 * x

            # Cálculo del error
            error = y - y_pred

            # Gradientes para descenso por gradiente
            grad_b0 = (-2 / n) * np.sum(error)
            grad_b1 = (-2 / n) * np.sum(error * x)

            # 6.5: actualización de parámetros usando alpha
            beta_0 -= alpha * grad_b0
            beta_1 -= alpha * grad_b1

            # 6.4: imprimir error cada cierto número de iteraciones
            if epoch % imprimir_error_cada == 0:
                mse = np.mean(error**2)
                print(f"Iteración {epoch} - Error (RMSE): {np.sqrt(mse):.4f}, beta_0: {(beta_0):.4f}, beta_1: {(beta_1):.4f}  ")

        return beta_0, beta_1



In [63]:
# ---------- DEFINICION Y USO DE LA CLASE ----------
if __name__ == "__main__":
    # Crear una instancia de la clase
    ruta_archivo = r"C:\PythonWs\Proyecto\proyecto_training_data.npy"
    analizador = AnalizadorDatos(ruta_archivo)

    # Mostrar las primeras filas
    analizador.mostrar_cabecera(10)

    # Separar los datos en entrenamiento y validación
    analizador.dividir_datos()

Datos cargados y mezclados correctamente.
   SalePrice  OverallQual  1stFlrSF  TotRmsAbvGrd  YearBuilt  LotFrontage
0   154500.0          6.0    1068.0           6.0     1963.0         70.0
1   325000.0          8.0    1500.0           9.0     1994.0         98.0
2   115000.0          5.0    1028.0           5.0     1927.0         56.0
3   159000.0          6.0    1004.0           7.0     1947.0         50.0
4   315500.0          9.0    1620.0           6.0     2007.0         89.0
5    75500.0          4.0     630.0           3.0     1972.0         21.0
6   311500.0          7.0    1137.0           8.0     1939.0         69.0
7   146000.0          6.0     855.0           7.0     1978.0         24.0
8    84500.0          4.0     630.0           3.0     1970.0         21.0
9   135500.0          5.0     872.0           8.0     1955.0         59.0
Datos divididos en 1168 registros para entrenamiento y 292 para validación.


C:\Users\EC\AppData\Local\Temp\ipykernel_16108\70130815.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.df["LotFrontage"].fillna(mediana, inplace=True)


In [64]:

# Calcular y mostrar estadísticas descriptivas
estadisticas = analizador.calcular_estadisticas()
print(estadisticas)

                      Media    Maximo   Minimo     Rango  DesviacionEstandar
SalePrice     181081.876712  755000.0  34900.0  720100.0        81096.489736
OverallQual        6.095034      10.0      1.0       9.0            1.402802
1stFlrSF        1161.268836    4692.0    334.0    4358.0          393.372616
TotRmsAbvGrd       6.532534      14.0      2.0      12.0            1.626715
YearBuilt       1971.120719    2009.0   1872.0     137.0           30.266595
LotFrontage       70.049658     313.0     21.0     292.0           22.723041


In [ ]:
analizador.graficar_histogramas()

In [ ]:
# 5. Para cada variable independiente x :
analizador.analizar_correlaciones()

In [ ]:
# Si ya tienes tu dataframe dentro de la clase, fuera puedes hacer:
analizador.graficar_matriz_correlacion_personalizada()


## Variables seleccionadas 
# Variable dependiente = SalePrice
# Variable independiente 1 = OverallQual: (Coeficiente de correlacion 0.7910) 

In [ ]:
#6.1 Vector con la variable independiente x,
x = analizador.entrenamiento["OverallQual"].to_numpy()

#6.2 Vector con la variable dependiente y,
y = analizador.entrenamiento["SalePrice"].to_numpy()

beta_0, beta_1 = analizador.entrenar_modelo_lineal_simple(x, y, 1000, 100, 0.0001)
print(f"Modelo entrenado: y = {beta_0:.2f} + {beta_1:.2f} * x")


Iteración 100 - Error (RMSE): 101846.1305, beta_0: 2466.6530, beta_1: 16474.3968  
Iteración 200 - Error (RMSE): 66732.9900, beta_0: 3514.1539, beta_1: 23851.6069  
Iteración 300 - Error (RMSE): 57168.9531, beta_0: 3926.9647, beta_1: 27159.9057  
Iteración 400 - Error (RMSE): 55049.0398, beta_0: 4055.9520, beta_1: 28648.2921  
Iteración 500 - Error (RMSE): 54607.4743, beta_0: 4058.0482, beta_1: 29322.6769  
Iteración 600 - Error (RMSE): 54511.1123, beta_0: 4003.4443, beta_1: 29632.9744  
Iteración 700 - Error (RMSE): 54484.2285, beta_0: 3923.5346, beta_1: 29780.4178  
Iteración 800 - Error (RMSE): 54471.2758, beta_0: 3832.3609, beta_1: 29855.0129  
Iteración 900 - Error (RMSE): 54461.1247, beta_0: 3736.2035, beta_1: 29897.0166  
Iteración 1000 - Error (RMSE): 54451.5478, beta_0: 3637.8714, beta_1: 29924.4347  
Modelo entrenado: y = 3637.87 + 29924.43 * x
